In [7]:
import pandas as pd
import numpy as np
import time, os
import matplotlib.pyplot as plt 
import seaborn as sns
import json
import datetime as dt

In [8]:
sensors = pd.read_csv('D:\\Air_Quality_health_Project\\data\\location_sensors.csv')
measurement = pd.read_csv('D:\\Air_Quality_health_Project\\data\\measurements.csv')
cases = pd.read_excel('D:\\Air_Quality_health_Project\\data\\Disease_case.xlsx')
population = pd.read_excel('D:\\Air_Quality_health_Project\\data\\States_population.xlsx')

In [9]:
population['Population_millions'] = (population['Population_millions'].str.split(' ').str[0]).astype(float)

In [10]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   State                36 non-null     object 
 1   Population_millions  36 non-null     float64
dtypes: float64(1), object(1)
memory usage: 708.0+ bytes


In [11]:
measurement.head()

,Sensor_id,Value,DateTimeFrom,DateTimeTo
0,14969,1070.0,2020-01-01,2020-02-01
1,14969,1010.0,2020-02-01,2020-03-01
2,14969,656.0,2020-03-01,2020-04-01
3,14969,110.0,2020-04-01,2020-05-01
4,14969,174.0,2020-05-01,2020-06-01


In [12]:
measurement['DateTimeFrom'] = pd.to_datetime(measurement['DateTimeFrom'])
measurement['DateTimeTo'] = pd.to_datetime(measurement['DateTimeTo'])

In [13]:
measurement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11684 entries, 0 to 11683
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Sensor_id     11684 non-null  int64         
 1   Value         11684 non-null  float64       
 2   DateTimeFrom  11684 non-null  datetime64[ns]
 3   DateTimeTo    11684 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1)
memory usage: 365.3 KB


In [14]:
measurement['Month'] = measurement['DateTimeFrom'].dt.month_name()

In [15]:
measurement['Pollutant_value'] = measurement['Value']

In [16]:
measurement = measurement.drop("Value", axis=1)

In [17]:
sensors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1145 entries, 0 to 1144
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Location_id  1145 non-null   int64  
 1   Sensor_id    1145 non-null   int64  
 2   State        1145 non-null   object 
 3   Area         1145 non-null   object 
 4   Sensor_name  1145 non-null   object 
 5   Latitude     1145 non-null   float64
 6   Longitude    1145 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 62.7+ KB


In [18]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             288 non-null    object 
 1   Month             288 non-null    object 
 2   Asthama_Cases     288 non-null    float64
 3   Bronchitis_Cases  288 non-null    int64  
 4   Heart_attacks     288 non-null    float64
 5   COPD_Cases        288 non-null    int64  
 6   Neumonia_Cases    288 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 15.9+ KB


In [19]:
numeric = cases.select_dtypes(['int64', 'float64']).columns
cases[numeric] = cases[numeric].round().astype('int64')

In [20]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   State             288 non-null    object
 1   Month             288 non-null    object
 2   Asthama_Cases     288 non-null    int64 
 3   Bronchitis_Cases  288 non-null    int64 
 4   Heart_attacks     288 non-null    int64 
 5   COPD_Cases        288 non-null    int64 
 6   Neumonia_Cases    288 non-null    int64 
dtypes: int64(5), object(2)
memory usage: 15.9+ KB


In [21]:
measurement.shape

(11684, 5)

In [22]:
sensor_cases = sensors.merge(population, on= 'State', how= 'left')

In [23]:
sensor_cases = sensor_cases.merge(cases, on= 'State')

In [24]:
sensor_cases.head()

,Location_id,Sensor_id,State,Area,Sensor_name,Latitude,Longitude,Population_millions,Month,Asthama_Cases,Bronchitis_Cases,Heart_attacks,COPD_Cases,Neumonia_Cases
0,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197,54.0,January,6400,11016,1150,16300,4550
1,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197,54.0,February,6400,10098,1100,14940,3900
2,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197,54.0,March,7200,8262,1200,13590,3900
3,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197,54.0,April,5600,6426,1300,10860,3900
4,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197,54.0,May,4800,5508,1250,9505,4550


In [25]:
measurement.shape, sensor_cases.shape

((11684, 5), (13740, 14))

In [26]:
Data = sensor_cases.merge(measurement, on= ['Sensor_id', 'Month'], how='left')

In [27]:
Data.head()

,Location_id,Sensor_id,State,Area,Sensor_name,Latitude,Longitude,Population_millions,Month,Asthama_Cases,Bronchitis_Cases,Heart_attacks,COPD_Cases,Neumonia_Cases,DateTimeFrom,DateTimeTo,Pollutant_value
0,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197,54.0,January,6400,11016,1150,16300,4550,2020-01-01,2020-02-01,1070.0
1,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197,54.0,February,6400,10098,1100,14940,3900,2020-02-01,2020-03-01,1010.0
2,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197,54.0,March,7200,8262,1200,13590,3900,2020-03-01,2020-04-01,656.0
3,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197,54.0,April,5600,6426,1300,10860,3900,2020-04-01,2020-05-01,110.0
4,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197,54.0,May,4800,5508,1250,9505,4550,2020-05-01,2020-06-01,174.0


In [28]:
Data.dropna(axis=0, inplace=True)

In [29]:
Data.loc[Data['Pollutant_value'] < 0, ['Pollutant_value']] = 0

In [30]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11574 entries, 0 to 13739
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Location_id          11574 non-null  int64         
 1   Sensor_id            11574 non-null  int64         
 2   State                11574 non-null  object        
 3   Area                 11574 non-null  object        
 4   Sensor_name          11574 non-null  object        
 5   Latitude             11574 non-null  float64       
 6   Longitude            11574 non-null  float64       
 7   Population_millions  11574 non-null  float64       
 8   Month                11574 non-null  object        
 9   Asthama_Cases        11574 non-null  int64         
 10  Bronchitis_Cases     11574 non-null  int64         
 11  Heart_attacks        11574 non-null  int64         
 12  COPD_Cases           11574 non-null  int64         
 13  Neumonia_Cases       11574 non-null 

In [31]:
Data.State.value_counts()

State
Delhi (NCT)       2056
Haryana           1568
Maharashtra       1163
Uttar Pradesh     1003
Karnataka          990
Madhya Pradesh     773
West Bengal        661
Rajasthan          540
Bihar              528
Kerala             460
Punjab             420
Telangana          303
Tamil Nadu         240
Andhra Pradesh     240
Gujarat            240
Odisha             120
Assam               65
Meghalaya           60
Jharkhand           55
Mizoram             50
Nagaland            20
Tripura             10
Chhattisgarh         9
Name: count, dtype: int64

In [32]:
Data.Sensor_name.value_counts()

Sensor_name
pm25 µg/m³    2402
no2 µg/m³     2385
co µg/m³      2346
so2 µg/m³     2282
pm10 µg/m³    2159
Name: count, dtype: int64

In [33]:
Data.Month.value_counts()

Month
December     1114
November     1069
September     969
October       964
August        957
July          952
June          951
April         941
March         941
May           937
February      904
January       875
Name: count, dtype: int64

In [34]:
Data.describe()

,Location_id,Sensor_id,Latitude,Longitude,Population_millions,Asthama_Cases,Bronchitis_Cases,Heart_attacks,COPD_Cases,Neumonia_Cases,DateTimeFrom,DateTimeTo,Pollutant_value
count,11574.000000,11574.000000,11574.000000,11574.000000,11574.000000,11574.000000,11574.000000,11574.000000,11574.000000,11574.000000,11574,11574,11574.000000
mean,7430.165371,23112.287368,23.677655,78.272416,76.448505,42682.631415,31547.336703,7707.003542,10747.361759,2862.057024,2020-06-22 05:30:12.130637568,2020-07-22 17:36:17.916018688,224.880030
min,301.000000,710.000000,8.514909,72.591912,1.300000,456.000000,313.000000,779.000000,230.000000,45.000000,2020-01-01 00:00:00,2020-02-01 00:00:00,0.000000
25%,5633.000000,15117.500000,19.137500,76.281342,29.000000,12000.000000,1380.000000,1236.000000,4200.000000,880.000000,2020-04-01 00:00:00,2020-05-01 00:00:00,16.100000
50%,6956.000000,20071.000000,25.619651,77.237738,70.000000,22000.000000,10200.000000,2500.000000,9300.000000,1400.000000,2020-07-01 00:00:00,2020-08-01 00:00:00,43.850000
75%,8475.000000,34821.000000,28.636429,78.451437,100.000000,60000.000000,34391.000000,11636.000000,15000.000000,2400.000000,2020-10-01 00:00:00,2020-11-01 00:00:00,161.000000
max,42240.000000,234638.000000,31.620000,94.098988,235.000000,231000.000000,276000.000000,41500.000000,29900.000000,18500.000000,2020-12-01 00:00:00,2021-01-01 00:00:00,34400.000000
std,2328.680279,10363.601012,5.995684,4.191024,61.191818,44805.411092,53720.644530,9880.211880,7513.992708,4084.442640,NaN,NaN,559.001208


In [35]:
Data.describe(exclude=['int64', 'float64', 'datetime64[ns]'])

,State,Area,Sensor_name,Month
count,11574,11574,11574,11574
unique,23,231,5,12
top,Delhi (NCT),Shastri Nagar,pm25 µg/m³,December
freq,2056,120,2402,1114


In [101]:
def get_aqi_category(pollutant, value):
    if pd.isnull(value) or pd.isnull(pollutant):
        return 'Unknown'
    try:
        if pollutant == 'pm25 µg/m³':
            if value <= 30: return 'Good'
            elif value <= 60: return 'Satisfactory'
            elif value <= 90: return 'Moderate'
            elif value <= 120: return 'Poor'
            elif value <= 250: return 'Very Poor'
            else: return 'Severe'
            
        elif pollutant == 'pm10 µg/m³':
            if value <= 50: return 'Good'
            elif value <= 100: return 'Satisfactory'
            elif value <= 250: return 'Moderate'
            elif value <= 350: return 'Poor'
            elif value <= 430: return 'Very Poor'
            else: return 'Severe'
            
        elif pollutant == 'no2 µg/m³':
            if value <= 40: return 'Good'
            elif value <= 80: return 'Satisfactory'
            elif value <= 180: return 'Moderate'
            elif value <= 280: return 'Poor'
            elif value <= 400: return 'Very Poor'
            else: return 'Severe'
            
        elif pollutant == 'so2 µg/m³':
            if value <= 40: return 'Good'
            elif value <= 80: return 'Satisfactory'
            elif value <= 380: return 'Moderate'
            elif value <= 800: return 'Poor'
            elif value <= 1600: return 'Very Poor'
            else: return 'Severe'
            
        elif pollutant == 'co µg/m³':
            if value <= 1: return 'Good'
            elif value <= 2: return 'Satisfactory'
            elif value <= 10: return 'Moderate'
            elif value <= 17: return 'Poor'
            elif value <= 34: return 'Very Poor'
            else: return 'Severe'
    except:        
        return 'Unknown'


In [102]:
Data['AQI_category'] =  Data.apply(lambda X: get_aqi_category(X['Sensor_name'], X['Pollutant_value']),axis= 1)

In [85]:
Data_pivot = Data.pivot_table(index=['State', 'Month'],
                 columns='Sensor_name',
                 values='Pollutant_value',
                 aggfunc='mean').reset_index()

In [111]:
Data_pivot = Data_pivot.round(2)

In [86]:
Data_pivot = Data_pivot.merge(cases, on= ["State", 'Month'], how="left")

In [112]:
Data_pivot.head()

,State,Month,co µg/m³,no2 µg/m³,pm10 µg/m³,pm25 µg/m³,so2 µg/m³,Asthama_Cases,Bronchitis_Cases,Heart_attacks,COPD_Cases,Neumonia_Cases
0,Andhra Pradesh,April,278.55,11.56,38.25,16.08,13.26,5600,6426,1300,10860,3900
1,Andhra Pradesh,August,488.25,12.76,39.05,14.43,3882.38,4800,5508,1200,10860,9100
2,Andhra Pradesh,December,895.75,23.82,124.20,71.62,103.28,7200,10098,1200,17655,3900
3,Andhra Pradesh,February,694.25,25.75,72.18,37.45,19.15,6400,10098,1100,14940,3900
4,Andhra Pradesh,January,706.50,24.70,83.60,47.92,15.76,6400,11016,1150,16300,4550


In [113]:
Data_pivot.head()

,State,Month,co µg/m³,no2 µg/m³,pm10 µg/m³,pm25 µg/m³,so2 µg/m³,Asthama_Cases,Bronchitis_Cases,Heart_attacks,COPD_Cases,Neumonia_Cases
0,Andhra Pradesh,April,278.55,11.56,38.25,16.08,13.26,5600,6426,1300,10860,3900
1,Andhra Pradesh,August,488.25,12.76,39.05,14.43,3882.38,4800,5508,1200,10860,9100
2,Andhra Pradesh,December,895.75,23.82,124.20,71.62,103.28,7200,10098,1200,17655,3900
3,Andhra Pradesh,February,694.25,25.75,72.18,37.45,19.15,6400,10098,1100,14940,3900
4,Andhra Pradesh,January,706.50,24.70,83.60,47.92,15.76,6400,11016,1150,16300,4550


In [114]:
Data.to_csv('D:\\Air_Quality_health_Project\\data\\clean_data\\AQI_Cases_data.csv', index=False)
Data_pivot.to_csv('D:\\Air_Quality_health_Project\\data\\clean_data\\Data_pivot.csv', index=False)